In [4]:
import pandas as pd
import numpy as np
import os

In [44]:
PATH_HTS_PERSONS_12_16_SA1_V1 = "inputs/melbourne/hts_12_16/P_VISTA12_16_SA1_V1.csv"
PATH_HTS_HOUSEHOLDS_12_16_SA1_V1 = "inputs/melbourne/hts_12_16/H_VISTA12_16_SA1_V1.csv"
GEO_FILE = "inputs/melbourne/geography/MB_2016_VIC.csv"
mapping_file = 'inputs/melbourne/geography/sa1_2011_2016.xlsx'
PERSONS_ATTRIBUTES = [
        "PERSID",
        "AGE",
        "SEX",
        "FULLTIMEWORK",
        "PARTTIMEWORK",
        "CASUALWORK",
        "ANYWORK",
        "HHID",
        "ReportingPeriod",
        "RP_ADPERSWGT_SA3", #Person weight for the given year, not used for anything for now
        "CW_ADPERSWGT_SA3",
        "HomeSA1"
    ]
HOUSEHOLDS_ATTRIBUTES = [
    "HHID", 
    "HHSIZE", 
    "CARS", 
    "TOTALVEHS", 
    "ReportingPeriod", 
    "RP_ADHHWGT_SA3",
    "CW_ADHHWGT_SA3",
    "HomeSA1"
    ]

In [45]:
def mapping_SA1_to_rest(df_target, geo_df):
    zip_from_geo = zip(
        geo_df['SA1_MAINCODE_2016'],
        geo_df['SA1_7DIGITCODE_2016'],
        geo_df['SA2_MAINCODE_2016'],
        geo_df['SA3_CODE_2016'],
        geo_df['SA4_CODE_2016']
    )

    dict_map = {}
    for sa1_full, sa1_7digit, sa2, sa3, sa4 in zip_from_geo:
        dict_map[sa1_full] = {
            'SA1': sa1_7digit,
            'SA2': sa2,
            'SA3': sa3,
            'SA4': sa4
        }
    SA1_arr, SA2_arr, SA3_arr, SA4_arr = [], [], [], []
    for sa1_full in df_target['HomeSA1']:
        if sa1_full in dict_map:
            val = dict_map[sa1_full]
            SA1_arr.append(val['SA1'])
            SA2_arr.append(val['SA2'])
            SA3_arr.append(val['SA3'])
            SA4_arr.append(val['SA4'])
        else:
            SA1_arr.append(None)
            SA2_arr.append(None)
            SA3_arr.append(None)
            SA4_arr.append(None)

    df_target['SA1'] = SA1_arr
    df_target['SA2'] = SA2_arr
    df_target['SA3'] = SA3_arr
    df_target['SA4'] = SA4_arr
    return df_target

In [46]:
geo_df = pd.read_csv(GEO_FILE)

df_map_raw  = pd.read_excel(mapping_file)

In [47]:
def is_unique_att(df, unique_att):
    N = df.shape[0]
    test_N = len(df[unique_att].unique())
    return True if N == test_N else False

In [51]:
h_original_df = pd.read_csv(PATH_HTS_HOUSEHOLDS_12_16_SA1_V1)
h_test_seed = h_original_df[HOUSEHOLDS_ATTRIBUTES]

if not is_unique_att(h_test_seed, "HHID"):
    raise Exception("THE ID OF THE HOUSEHOLD IS NOT UNIQUE")

/var/folders/fg/wfh8sp4x7lbb49hgnww7yrlm0000gn/T/ipykernel_53152/3413693049.py:1: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  h_original_df = pd.read_csv(PATH_HTS_HOUSEHOLDS_12_16_SA1_V1)


In [52]:
h_test_seed["CARS"] = np.where(h_test_seed["CARS"] == 0, "No", "Yes")
h_test_seed = h_test_seed.reset_index(drop=True)
h_test_seed['hhnum'] = h_test_seed.index + 1
h_test_seed

/var/folders/fg/wfh8sp4x7lbb49hgnww7yrlm0000gn/T/ipykernel_53152/1163028317.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_test_seed["CARS"] = np.where(h_test_seed["CARS"] == 0, "No", "Yes")


,HHID,HHSIZE,CARS,TOTALVEHS,ReportingPeriod,RP_ADHHWGT_SA3,CW_ADHHWGT_SA3,HomeSA1,hhnum
0,Y12H0000101,4,Yes,2,2012-14,161.52,91.34,20904122028,1
1,Y12H0000102,4,Yes,3,2012-14,162.05,91.26,20904122028,2
2,Y12H0000103,4,Yes,2,2012-14,158.73,91.03,20904122028,3
3,Y12H0000104,3,Yes,3,2012-14,162.45,91.81,20904122028,4
4,Y12H0000107,3,Yes,1,2012-14,196.58,107.80,20904122028,5
...,...,...,...,...,...,...,...,...,...
18147,Y16H2080525,6,No,2,2014-16,441.81,230.15,21304135805,18148
18148,Y16H2080531,1,No,1,2014-16,443.60,228.19,21304135805,18149
18149,Y16H2080532,2,Yes,1,2014-16,441.81,230.15,21304135805,18150
18150,Y16H2080533,1,No,0,2014-16,448.61,231.13,21304135805,18151


In [53]:
dict_map = {}
for name_2011, name_2016 in zip(df_map_raw['SA1_MAINCODE_2011'], df_map_raw['SA1_MAINCODE_2016']):
    dict_map[name_2011] = name_2016
    
h_test_seed['HomeSA1'].replace(dict_map, inplace=True)
h_test_seed

,HHID,HHSIZE,CARS,TOTALVEHS,ReportingPeriod,RP_ADHHWGT_SA3,CW_ADHHWGT_SA3,HomeSA1,hhnum
0,Y12H0000101,4,Yes,2,2012-14,161.52,91.34,20904122028,1
1,Y12H0000102,4,Yes,3,2012-14,162.05,91.26,20904122028,2
2,Y12H0000103,4,Yes,2,2012-14,158.73,91.03,20904122028,3
3,Y12H0000104,3,Yes,3,2012-14,162.45,91.81,20904122028,4
4,Y12H0000107,3,Yes,1,2012-14,196.58,107.80,20904122028,5
...,...,...,...,...,...,...,...,...,...
18147,Y16H2080525,6,No,2,2014-16,441.81,230.15,21304135805,18148
18148,Y16H2080531,1,No,1,2014-16,443.60,228.19,21304135805,18149
18149,Y16H2080532,2,Yes,1,2014-16,441.81,230.15,21304135805,18150
18150,Y16H2080533,1,No,0,2014-16,448.61,231.13,21304135805,18151


In [54]:
h_test_seed = mapping_SA1_to_rest(h_test_seed, geo_df)

In [55]:
h_test_seed

,HHID,HHSIZE,CARS,TOTALVEHS,ReportingPeriod,RP_ADHHWGT_SA3,CW_ADHHWGT_SA3,HomeSA1,hhnum,SA1,SA2,SA3,SA4
0,Y12H0000101,4,Yes,2,2012-14,161.52,91.34,20904122028,1,2122028,209041220,20904,209
1,Y12H0000102,4,Yes,3,2012-14,162.05,91.26,20904122028,2,2122028,209041220,20904,209
2,Y12H0000103,4,Yes,2,2012-14,158.73,91.03,20904122028,3,2122028,209041220,20904,209
3,Y12H0000104,3,Yes,3,2012-14,162.45,91.81,20904122028,4,2122028,209041220,20904,209
4,Y12H0000107,3,Yes,1,2012-14,196.58,107.80,20904122028,5,2122028,209041220,20904,209
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18147,Y16H2080525,6,No,2,2014-16,441.81,230.15,21304135805,18148,2135805,213041358,21304,213
18148,Y16H2080531,1,No,1,2014-16,443.60,228.19,21304135805,18149,2135805,213041358,21304,213
18149,Y16H2080532,2,Yes,1,2014-16,441.81,230.15,21304135805,18150,2135805,213041358,21304,213
18150,Y16H2080533,1,No,0,2014-16,448.61,231.13,21304135805,18151,2135805,213041358,21304,213


In [64]:
h_test_seed.to_csv('outputs/seed_households.csv')

In [16]:
p_original_df = pd.read_csv(PATH_HTS_PERSONS_12_16_SA1_V1)
p_test_seed = p_original_df[PERSONS_ATTRIBUTES]

if not is_unique_att(p_test_seed, "PERSID"):
    raise Exception("THE ID OF PERSON IS NOT UNIQUE")

In [56]:
p_test_seed.loc[p_test_seed["CASUALWORK"] == "Yes", "ANYWORK"] = "CASUALWORK"
p_test_seed.loc[p_test_seed["PARTTIMEWORK"] == "Yes", "ANYWORK"] = "PARTTIMEWORK"
p_test_seed.loc[p_test_seed["FULLTIMEWORK"] == "Yes", "ANYWORK"] = "FULLTIMEWORK"
p_test_seed = p_test_seed.drop(
    columns=["FULLTIMEWORK", "PARTTIMEWORK", "CASUALWORK"]
)

In [58]:
src_h, target_h = 'HHID', 'hhnum'
hold = h_test_seed.set_index(src_h)[target_h].drop_duplicates()
p_test_seed['hhnum'] = p_test_seed[src_h].map(hold)

In [59]:
p_test_seed

,PERSID,AGE,SEX,ANYWORK,HHID,ReportingPeriod,RP_ADPERSWGT_SA3,CW_ADPERSWGT_SA3,HomeSA1,hhnum
0,Y12H0000101P01,50,Male,FULLTIMEWORK,Y12H0000101,2012-14,149.58,81.70,20904122028,1
1,Y12H0000101P02,43,Female,No,Y12H0000101,2012-14,170.04,90.70,20904122028,1
2,Y12H0000101P03,11,Female,No,Y12H0000101,2012-14,183.96,98.05,20904122028,1
3,Y12H0000101P04,6,Female,No,Y12H0000101,2012-14,183.34,93.07,20904122028,1
4,Y12H0000102P01,57,Female,No,Y12H0000102,2012-14,146.42,78.92,20904122028,2
...,...,...,...,...,...,...,...,...,...,...
46557,Y16H2080531P01,54,Male,FULLTIMEWORK,Y16H2080531,2014-16,397.82,198.81,21304135805,18149
46558,Y16H2080532P01,79,Female,No,Y16H2080532,2014-16,463.51,236.78,21304135805,18150
46559,Y16H2080532P02,58,Male,No,Y16H2080532,2014-16,396.22,200.53,21304135805,18150
46560,Y16H2080533P01,66,Female,No,Y16H2080533,2014-16,379.88,192.30,21304135805,18151


In [60]:
p_test_seed['HomeSA1'].replace(dict_map, inplace=True)
p_test_seed

,PERSID,AGE,SEX,ANYWORK,HHID,ReportingPeriod,RP_ADPERSWGT_SA3,CW_ADPERSWGT_SA3,HomeSA1,hhnum
0,Y12H0000101P01,50,Male,FULLTIMEWORK,Y12H0000101,2012-14,149.58,81.70,20904122028,1
1,Y12H0000101P02,43,Female,No,Y12H0000101,2012-14,170.04,90.70,20904122028,1
2,Y12H0000101P03,11,Female,No,Y12H0000101,2012-14,183.96,98.05,20904122028,1
3,Y12H0000101P04,6,Female,No,Y12H0000101,2012-14,183.34,93.07,20904122028,1
4,Y12H0000102P01,57,Female,No,Y12H0000102,2012-14,146.42,78.92,20904122028,2
...,...,...,...,...,...,...,...,...,...,...
46557,Y16H2080531P01,54,Male,FULLTIMEWORK,Y16H2080531,2014-16,397.82,198.81,21304135805,18149
46558,Y16H2080532P01,79,Female,No,Y16H2080532,2014-16,463.51,236.78,21304135805,18150
46559,Y16H2080532P02,58,Male,No,Y16H2080532,2014-16,396.22,200.53,21304135805,18150
46560,Y16H2080533P01,66,Female,No,Y16H2080533,2014-16,379.88,192.30,21304135805,18151


In [61]:
p_test_seed = mapping_SA1_to_rest(p_test_seed, geo_df)

In [62]:
p_test_seed

,PERSID,AGE,SEX,ANYWORK,HHID,ReportingPeriod,RP_ADPERSWGT_SA3,CW_ADPERSWGT_SA3,HomeSA1,hhnum,SA1,SA2,SA3,SA4
0,Y12H0000101P01,50,Male,FULLTIMEWORK,Y12H0000101,2012-14,149.58,81.70,20904122028,1,2122028,209041220,20904,209
1,Y12H0000101P02,43,Female,No,Y12H0000101,2012-14,170.04,90.70,20904122028,1,2122028,209041220,20904,209
2,Y12H0000101P03,11,Female,No,Y12H0000101,2012-14,183.96,98.05,20904122028,1,2122028,209041220,20904,209
3,Y12H0000101P04,6,Female,No,Y12H0000101,2012-14,183.34,93.07,20904122028,1,2122028,209041220,20904,209
4,Y12H0000102P01,57,Female,No,Y12H0000102,2012-14,146.42,78.92,20904122028,2,2122028,209041220,20904,209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46557,Y16H2080531P01,54,Male,FULLTIMEWORK,Y16H2080531,2014-16,397.82,198.81,21304135805,18149,2135805,213041358,21304,213
46558,Y16H2080532P01,79,Female,No,Y16H2080532,2014-16,463.51,236.78,21304135805,18150,2135805,213041358,21304,213
46559,Y16H2080532P02,58,Male,No,Y16H2080532,2014-16,396.22,200.53,21304135805,18150,2135805,213041358,21304,213
46560,Y16H2080533P01,66,Female,No,Y16H2080533,2014-16,379.88,192.30,21304135805,18151,2135805,213041358,21304,213


In [63]:
p_test_seed.to_csv('outputs/seed_persons.csv')

In [67]:
p_test_seed

,PERSID,AGE,SEX,ANYWORK,HHID,ReportingPeriod,RP_ADPERSWGT_SA3,CW_ADPERSWGT_SA3,HomeSA1,hhnum,SA1,SA2,SA3,SA4
0,Y12H0000101P01,50,Male,FULLTIMEWORK,Y12H0000101,2012-14,149.58,81.70,20904122028,1,2122028,209041220,20904,209
1,Y12H0000101P02,43,Female,No,Y12H0000101,2012-14,170.04,90.70,20904122028,1,2122028,209041220,20904,209
2,Y12H0000101P03,11,Female,No,Y12H0000101,2012-14,183.96,98.05,20904122028,1,2122028,209041220,20904,209
3,Y12H0000101P04,6,Female,No,Y12H0000101,2012-14,183.34,93.07,20904122028,1,2122028,209041220,20904,209
4,Y12H0000102P01,57,Female,No,Y12H0000102,2012-14,146.42,78.92,20904122028,2,2122028,209041220,20904,209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46557,Y16H2080531P01,54,Male,FULLTIMEWORK,Y16H2080531,2014-16,397.82,198.81,21304135805,18149,2135805,213041358,21304,213
46558,Y16H2080532P01,79,Female,No,Y16H2080532,2014-16,463.51,236.78,21304135805,18150,2135805,213041358,21304,213
46559,Y16H2080532P02,58,Male,No,Y16H2080532,2014-16,396.22,200.53,21304135805,18150,2135805,213041358,21304,213
46560,Y16H2080533P01,66,Female,No,Y16H2080533,2014-16,379.88,192.30,21304135805,18151,2135805,213041358,21304,213


In [74]:
def remap_SA4(output_dir, export_csv=False):
    new_map_SA4 = {
        220: [203, 217],
        221: [213, 201, 215],
        222: [210, 202],
        223: [209, 204, 216],
        224: [212, 205]
    }
    aff_sa4 = []
    for k in new_map_SA4: aff_sa4.extend(new_map_SA4[k])

    file_control_SA4 = 'controls_total_SA4.csv'
    file_hh = 'seed_households.csv'
    file_p = 'seed_persons.csv'
    file_geo = 'geo_cross_walk.csv'

    df_hh = pd.read_csv(os.path.join(output_dir, file_hh))
    df_p = pd.read_csv(os.path.join(output_dir, file_p))
    df_SA4 = pd.read_csv(os.path.join(output_dir, file_control_SA4))
    df_geo = pd.read_csv(os.path.join(output_dir, file_geo))

    mapping_dict = {}
    for z in df_SA4['SA4_CODE_2016']:
        if z not in aff_sa4:
            mapping_dict[z] = z
        else:
            for k in new_map_SA4:
                if z in new_map_SA4[k]:
                    mapping_dict[z] = k
    
    df_hh = add_new_SA4_col(df_hh, "SA4", mapping_dict)
    df_hh = df_hh.drop(["SA4"], axis=1)

    df_p = add_new_SA4_col(df_p, "SA4", mapping_dict)
    df_p = df_p.drop(["SA4"], axis=1)

    df_geo = add_new_SA4_col(df_geo, "SA4_CODE_2016", mapping_dict)
    df_geo = df_geo.drop(["SA4_CODE_2016"], axis=1)
    df_geo = df_geo.drop(df_geo[df_geo['newSA4'] == 299].index)
    df_geo = df_geo.drop(df_geo[df_geo['newSA4'] == 297].index)

    df_SA4 = add_new_SA4_col(df_SA4, "SA4_CODE_2016", mapping_dict)
    df_SA4 = df_SA4.drop(["SA4_CODE_2016"], axis=1)
    df_SA4 = df_SA4.groupby(['newSA4']).sum()
    df_SA4["SA4"] = df_SA4.index
    
def add_new_SA4_col(df, name_old_col, map_di, name_new_col="newSA4"):
    arr = []
    for old_z in df[name_old_col]:
        if old_z in map_di: arr.append(map_di[old_z])
        else: log.warning(f"this {old_z} is not exist in mapping dict, probably type convert issue")
    df[name_new_col] = arr
    return df

In [75]:
remap_SA4('outputs/',export_csv=True)